<a href="https://colab.research.google.com/github/ravimashru/100-days-of-deep-learning/blob/master/docs/days/027_Custom_Models_Losses_and_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Models, Losses and Metrics

In [1]:
import tensorflow as tf
from tensorflow import keras

print('TensorFlow: ', tf.__version__)
print('Keras: ', keras.__version__)

TensorFlow:  2.3.0
Keras:  2.4.0


In [2]:
from keras.datasets import mnist
from sklearn.model_selection import train_test_split

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train)

print('Training set: ', X_train.shape, y_train.shape)
print('Validation set: ', X_valid.shape, y_valid.shape)
print('Test set: ', X_test.shape, y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
Training set:  (45000, 28, 28) (45000,)
Validation set:  (15000, 28, 28) (15000,)
Test set:  (10000, 28, 28) (10000,)


In [150]:
class ResidualLayer(keras.layers.Layer):
  def __init__(self, n_layers, n_neurons, **kwargs):
    super().__init__(**kwargs)
    self.hidden = [
      keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal")
      for _ in range(n_layers)
    ]

  def call(self, inputs):
    Z = inputs
    for layer in self.hidden:
      Z = layer(Z)
    return inputs + Z

class ResidualModel(keras.Model):
  def __init__(self, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.flatten = keras.layers.Flatten(dtype=tf.float32)
    self.hidden = keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal")
    self.residual = ResidualLayer(3, 128)
    self.out = keras.layers.Dense(output_dim)
    self.reconstruct = keras.layers.Dense(784)

  def call(self, inputs):
    flat_inputs = self.flatten(inputs)
    Z = self.hidden(flat_inputs)
    recon = self.reconstruct(Z)
    recon_loss = 0.05 * tf.reduce_mean(tf.square(recon - tf.cast(flat_inputs, dtype=tf.float32)))
    self.add_loss(recon_loss)
    self.add_metric(recon_loss, name='recon_loss', aggregation='mean')
    for _ in range(3):
      Z = self.residual(Z)
    return self.out(Z)

In [151]:
model = ResidualModel(10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [152]:
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
352/352 [==============================] - 4s 11ms/step - loss: 106.0920 - accuracy: 0.1185 - recon_loss: 92.3067 - val_loss: 55.8659 - val_accuracy: 0.1155 - val_recon_loss: 42.0017
Epoch 2/10
352/352 [==============================] - 4s 11ms/step - loss: 47.9580 - accuracy: 0.1143 - recon_loss: 34.1907 - val_loss: 43.0808 - val_accuracy: 0.1117 - val_recon_loss: 29.2217
Epoch 3/10
352/352 [==============================] - 4s 11ms/step - loss: 40.5101 - accuracy: 0.1122 - recon_loss: 26.7454 - val_loss: 38.9966 - val_accuracy: 0.1102 - val_recon_loss: 25.1391
Epoch 4/10
352/352 [==============================] - 4s 11ms/step - loss: 37.6594 - accuracy: 0.1113 - recon_loss: 23.8963 - val_loss: 37.1362 - val_accuracy: 0.1087 - val_recon_loss: 23.2797
Epoch 5/10
352/352 [==============================] - 4s 11ms/step - loss: 36.3487 - accuracy: 0.1103 - recon_loss: 22.5860 - val_loss: 35.9400 - val_accuracy: 0.1075 - val_recon_loss: 22.0822
Epoch 6/10
352/352 [==============